## Import libraries

In [378]:
import numpy as np
import pandas as pd
import statsmodels as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
import itertools

In [379]:
import statsmodels

In [380]:
statsmodels.__version__

'0.9.0'

## Path to data files

In [381]:
PATH = 'dataset'
PATH_TO_train_data = PATH + '/' + 'yds_train2018.csv'
PATH_TO_test_data = PATH + '/' + 'yds_test2018.csv'
PATH_TO_promotional_expense = PATH + '/' + 'promotional_expense.csv'
PATH_TO_holidays = PATH + '/' + 'holidays.xlsx'

In [382]:
train_data = pd.read_csv(PATH_TO_train_data)

## Drop Merchant_ID and S_No as they are niot required.

In [383]:
train_data.drop(columns=['Merchant_ID', 'S_No'], inplace=True)

In [384]:
train_data = train_data.groupby(['Year', 'Month', 'Product_ID', 'Country']).Sales.sum().reset_index()

In [385]:
train_data.head()

,Year,Month,Product_ID,Country,Sales
0,2013,1,1,Argentina,34346025.00
1,2013,1,1,Columbia,3947356.31
2,2013,1,2,Argentina,2751851.48
3,2013,1,2,Belgium,314850.13
4,2013,1,2,Columbia,2885137.06


## Find unique combinations of Country and Product_ID

In [386]:
find_unique_country_pro = pd.DataFrame({'Country': train_data.Country, 'Product_ID': train_data.Product_ID})

In [387]:
find_unique_country_pro.head()

,Country,Product_ID
0,Argentina,1
1,Columbia,1
2,Argentina,2
3,Belgium,2
4,Columbia,2


In [388]:
unique = find_unique_country_pro.drop_duplicates()
unique.sort_values(['Country'])

,Country,Product_ID
0,Argentina,1
2,Argentina,2
226,Argentina,3
3,Belgium,2
1,Columbia,1
4,Columbia,2
186,Columbia,3
5,Denmark,2
6,England,4
8,England,5


## Function for ARIMA model.
### Takes country, id and series data as input.
### It tries different values of pdq values and takes th best value out of them.
### Using those values, it predicts the next 36 data points.
### There are 11 different combinations. Apply sarimax on all combinations.

In [389]:
import sys
warnings.filterwarnings("ignore") # specify to ignore warning messages

def sarimax(series_data, country, id):
    # Define the p, d and q parameters to take any value between 0 and 2
    p = d = q = range(0, 4)

    # Generate all different combinations of p, d and q triplets
    pdq = list(itertools.product(p, d, q))

    # Generate all different combinations of seasonal p, q and q triplets
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
    
    
    best_aic = np.inf
    best_pdq = None
    best_seasonal_pdq = None
    temp_model = None

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                temp_model = sm.tsa.statespace.sarimax.SARIMAX(series_data, order=param, seasonal_order=param_seasonal)
                results = temp_model.fit()
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_pdq = param
                    best_seasonal_pdq = param_seasonal
            except:
                continue
    best_model = sm.tsa.statespace.sarimax.SARIMAX(series_data,
                                      order=best_pdq,
                                      seasonal_order=best_seasonal_pdq, enforce_invertibility=False)
    best_results = best_model.fit()
    n_steps = 36
    pred_uc_99 = best_results.get_forecast(steps=36, alpha=0.01)
    df = pred_uc_99.conf_int()
    
    df['forcast'] = df['lower Sales'] * 0.5 + df['upper Sales'] * 0.5
    df['Country'] = country
    df['Product_ID'] = id
    
    return best_pdq, best_seasonal_pdq, df
    print("Best SARIMAX{}x{}12 model - AIC:{}".format(best_pdq, best_seasonal_pdq, best_aic))

In [390]:
df.head()

,lower Sales,upper Sales,forcast,Country,Product_ID
2015-11-01,-3.698110e+07,8.919514e+07,2.610702e+07,Finland,4
2015-12-01,-4.127920e+07,1.708689e+08,6.479484e+07,Finland,4
2016-01-01,-1.021148e+08,2.396559e+08,6.877055e+07,Finland,4
2016-02-01,-1.068720e+08,3.688945e+08,1.310112e+08,Finland,4
2016-03-01,-2.806585e+08,3.515241e+08,3.543283e+07,Finland,4


In [391]:
df.shape

(36, 5)

In [392]:
x = pd.concat([df, df]).shape

In [393]:
argentina_1 = train_data.loc[train_data.Country == 'Argentina', :]
argentina_1 = argentina_1.sort_values(['Year', 'Month']).loc[argentina_1.Product_ID == 1]
argentina_1['Day'] = 1
argentina_1['Date'] = pd.to_datetime(argentina_1[['Year', 'Month', 'Day']])
argentina_1.head()

sales = argentina_1.Sales
sales.index = range(sales.count())
date = argentina_1.Date
date.index = range(date.count())
argentina_1_series = pd.DataFrame({'Date': date, 'Sales': sales})
argentina_1_series.head()

argentina_1_series.set_index('Date', inplace=True)

argentina_1_series = pd.Series(argentina_1_series.Sales, index=argentina_1_series.index)
argentina_1_series.head()

Date
2013-01-01    34346025.0
2013-02-01    32005575.0
2013-03-01    32530050.0
2013-04-01    35588700.0
2013-05-01    38789100.0
Name: Sales, dtype: float64

In [394]:
best_pdq, best_seasonal_pdq, df = sarimax(argentina_1_series, 'Argentina', 1)
main_df = df

In [395]:
main_df.shape

(36, 5)

In [396]:
argentina_2 = train_data.loc[train_data.Country == 'Argentina', :]
argentina_2 = argentina_2.sort_values(['Year', 'Month']).loc[argentina_2.Product_ID == 2]
argentina_2['Day'] = 1
argentina_2['Date'] = pd.to_datetime(argentina_2[['Year', 'Month', 'Day']])
argentina_2.head()

sales = argentina_2.Sales
sales.index = range(sales.count())
date = argentina_2.Date
date.index = range(date.count())
argentina_2_series = pd.DataFrame({'Date': date, 'Sales': sales})
argentina_2_series.head()

argentina_2_series.set_index('Date', inplace=True)

argentina_2_series = pd.Series(argentina_2_series.Sales, index=argentina_2_series.index)
argentina_2_series.head()

Date
2013-01-01    2751851.48
2013-02-01    2804313.12
2013-03-01    2573004.98
2013-04-01    3436237.42
2013-05-01    3772468.84
Name: Sales, dtype: float64

In [397]:
best_pdq, best_seasonal_pdq, df = sarimax(argentina_2_series, 'Argentina', 2)

In [398]:
main_df = pd.concat([main_df, df])

In [399]:
main_df.shape

(72, 5)

In [400]:
argentina_3 = train_data.loc[train_data.Country == 'Argentina', :]
argentina_3 = argentina_3.sort_values(['Year', 'Month']).loc[argentina_3.Product_ID == 3]
argentina_3['Day'] = 1
argentina_3['Date'] = pd.to_datetime(argentina_3[['Year', 'Month', 'Day']])
argentina_3.head()

sales = argentina_3.Sales
sales.index = range(sales.count())
date = argentina_3.Date
date.index = range(date.count())
argentina_3_series = pd.DataFrame({'Date': date, 'Sales': sales})
argentina_3_series.head()

argentina_3_series.set_index('Date', inplace=True)

argentina_3_series = pd.Series(argentina_3_series.Sales, index=argentina_3_series.index)
argentina_3_series.head()

Date
2015-01-01     126000.0
2015-02-01    4180050.0
2015-03-01    7437150.0
2015-04-01    7626150.0
2015-05-01    6189750.0
Name: Sales, dtype: float64

In [401]:
best_pdq, best_seasonal_pdq, df = sarimax(argentina_3_series, 'Argentina', 3)

In [402]:
main_df = pd.concat([main_df, df])

In [403]:
main_df.shape

(108, 5)

In [404]:
belgium_2 = train_data.loc[train_data.Country == 'Belgium', :]
belgium_2 = belgium_2.sort_values(['Year', 'Month']).loc[belgium_2.Product_ID == 2]
belgium_2['Day'] = 1
belgium_2['Date'] = pd.to_datetime(belgium_2[['Year', 'Month', 'Day']])
belgium_2.head()

sales = belgium_2.Sales
sales.index = range(sales.count())
date = belgium_2.Date
date.index = range(date.count())
belgium_2_series = pd.DataFrame({'Date': date, 'Sales': sales})
belgium_2_series.head()

belgium_2_series.set_index('Date', inplace=True)

belgium_2_series = pd.Series(belgium_2_series.Sales, index=belgium_2_series.index)
belgium_2_series.head()

Date
2013-01-01    314850.13
2013-02-01    435185.52
2013-03-01    268694.09
2013-04-01    461560.40
2013-05-01    464857.26
Name: Sales, dtype: float64

In [405]:
best_pdq, best_seasonal_pdq, df = sarimax(belgium_2_series, 'Belgium', 2)

In [406]:
main_df = pd.concat([main_df, df])

In [407]:
main_df.shape

(144, 5)

In [408]:
columbia_1 = train_data.loc[train_data.Country == 'Columbia', :]
columbia_1 = columbia_1.sort_values(['Year', 'Month']).loc[columbia_1.Product_ID == 1]
columbia_1['Day'] = 1
columbia_1['Date'] = pd.to_datetime(columbia_1[['Year', 'Month', 'Day']])
columbia_1.head()

sales = columbia_1.Sales
sales.index = range(sales.count())
date = columbia_1.Date
date.index = range(date.count())
columbia_1_series = pd.DataFrame({'Date': date, 'Sales': sales})
columbia_1_series.head()

columbia_1_series.set_index('Date', inplace=True)

columbia_1_series = pd.Series(columbia_1_series.Sales, index=columbia_1_series.index)
columbia_1_series.head()

Date
2013-01-01    3947356.31
2013-02-01    4074073.20
2013-03-01    4189422.98
2013-04-01    4229307.08
2013-05-01    4452229.12
Name: Sales, dtype: float64

In [ ]:
best_pdq, best_seasonal_pdq, df = sarimax(columbia_1_series, 'Columbia', 1)

In [249]:
main_df = pd.concat([main_df, df])

In [250]:
main_df.shape

(180, 5)

In [251]:
columbia_2 = train_data.loc[train_data.Country == 'Columbia', :]
columbia_2 = columbia_2.sort_values(['Year', 'Month']).loc[columbia_2.Product_ID == 2]
columbia_2['Day'] = 1
columbia_2['Date'] = pd.to_datetime(columbia_2[['Year', 'Month', 'Day']])
columbia_2.head()

sales = columbia_2.Sales
sales.index = range(sales.count())
date = columbia_2.Date
date.index = range(date.count())
columbia_2_series = pd.DataFrame({'Date': date, 'Sales': sales})
columbia_2_series.head()

columbia_2_series.set_index('Date', inplace=True)

columbia_2_series = pd.Series(columbia_2_series.Sales, index=columbia_2_series.index)
columbia_2_series.head()

Date
2013-01-01    2885137.06
2013-02-01    2481817.06
2013-03-01    2821691.18
2013-04-01    3149451.18
2013-05-01    3389234.12
Name: Sales, dtype: float64

In [252]:
best_pdq, best_seasonal_pdq, df = sarimax(columbia_2_series, 'Columbia', 2)

In [253]:
main_df = pd.concat([main_df, df])

In [254]:
main_df.shape

(216, 5)

In [255]:
columbia_3 = train_data.loc[train_data.Country == 'Columbia', :]
columbia_3 = columbia_3.sort_values(['Year', 'Month']).loc[columbia_3.Product_ID == 3]
columbia_3['Day'] = 1
columbia_3['Date'] = pd.to_datetime(columbia_3[['Year', 'Month', 'Day']])
columbia_3.head()

sales = columbia_3.Sales
sales.index = range(sales.count())
date = columbia_3.Date
date.index = range(date.count())
columbia_3_series = pd.DataFrame({'Date': date, 'Sales': sales})
columbia_3_series.head()

columbia_3_series.set_index('Date', inplace=True)

columbia_3_series = pd.Series(columbia_3_series.Sales, index=columbia_3_series.index)
columbia_3_series.head()

Date
2014-09-01     22459.60
2014-10-01    161523.85
2014-11-01    175611.00
2014-12-01    151235.65
2015-01-01    159065.70
Name: Sales, dtype: float64

In [256]:
best_pdq, best_seasonal_pdq, df = sarimax(columbia_3_series, 'Columbia', 3)

In [257]:
main_df = pd.concat([main_df, df])

In [258]:
main_df.shape

(252, 5)

In [259]:
denmark_2 = train_data.loc[train_data.Country == 'Denmark', :]
denmark_2 = denmark_2.sort_values(['Year', 'Month']).loc[denmark_2.Product_ID == 2]
denmark_2['Day'] = 1
denmark_2['Date'] = pd.to_datetime(denmark_2[['Year', 'Month', 'Day']])
denmark_2.head()

sales = denmark_2.Sales
sales.index = range(sales.count())
date = denmark_2.Date
date.index = range(date.count())
denmark_2_series = pd.DataFrame({'Date': date, 'Sales': sales})
denmark_2_series.head()

denmark_2_series.set_index('Date', inplace=True)

denmark_2_series = pd.Series(denmark_2_series.Sales, index=denmark_2_series.index)
denmark_2_series.head()

Date
2013-01-01    1.018476e+08
2013-02-01    8.445781e+07
2013-03-01    8.915608e+07
2013-04-01    9.916639e+07
2013-05-01    1.060422e+08
Name: Sales, dtype: float64

In [260]:
best_pdq, best_seasonal_pdq, df = sarimax(denmark_2_series, 'Denmark', 2)

In [261]:
main_df = pd.concat([main_df, df])

In [262]:
main_df.shape

(288, 5)

In [263]:
england_4 = train_data.loc[train_data.Country == 'England', :]
england_4 = england_4.sort_values(['Year', 'Month']).loc[england_4.Product_ID == 4]
england_4['Day'] = 1
england_4['Date'] = pd.to_datetime(england_4[['Year', 'Month', 'Day']])
england_4.head()

sales = england_4.Sales
sales.index = range(sales.count())
date = england_4.Date
date.index = range(date.count())
england_4_series = pd.DataFrame({'Date': date, 'Sales': sales})
england_4_series.head()

england_4_series.set_index('Date', inplace=True)

england_4_series = pd.Series(england_4_series.Sales, index=england_4_series.index)
england_4_series.head()

Date
2013-01-01    166907.57
2013-02-01    140685.22
2013-03-01    159316.90
2013-04-01    179295.95
2013-05-01    175741.65
Name: Sales, dtype: float64

In [264]:
best_pdq, best_seasonal_pdq, df = sarimax(england_4_series, 'England', 4)

In [265]:
main_df = pd.concat([main_df, df])

In [266]:
main_df.shape

(324, 5)

In [267]:
england_5 = train_data.loc[train_data.Country == 'England', :]
england_5 = england_5.sort_values(['Year', 'Month']).loc[england_5.Product_ID == 5]
england_5['Day'] = 1
england_5['Date'] = pd.to_datetime(england_5[['Year', 'Month', 'Day']])
england_5.head()

sales = england_5.Sales
sales.index = range(sales.count())
date = england_5.Date
date.index = range(date.count())
england_5_series = pd.DataFrame({'Date': date, 'Sales': sales})
england_5_series.head()

england_5_series.set_index('Date', inplace=True)

england_5_series = pd.Series(england_5_series.Sales, index=england_5_series.index)
england_5_series.head()

Date
2013-01-01    462370.62
2013-02-01    414931.16
2013-03-01    439883.54
2013-04-01    505948.04
2013-05-01    491075.38
Name: Sales, dtype: float64

In [268]:
best_pdq, best_seasonal_pdq, df = sarimax(england_5_series, 'England', 5)

In [269]:
main_df = pd.concat([main_df, df])

In [270]:
main_df.shape

(360, 5)

In [271]:
finland_4 = train_data.loc[train_data.Country == 'Finland', :]
finland_4 = finland_4.sort_values(['Year', 'Month']).loc[finland_4.Product_ID == 4]
finland_4['Day'] = 1
finland_4['Date'] = pd.to_datetime(finland_4[['Year', 'Month', 'Day']])
finland_4.head()

sales = finland_4.Sales
sales.index = range(sales.count())
date = finland_4.Date
date.index = range(date.count())
finland_4_series = pd.DataFrame({'Date': date, 'Sales': sales})

finland_4_series.set_index('Date', inplace=True)

finland_4_series = pd.Series(finland_4_series.Sales, index=finland_4_series.index)
finland_4_series.head()

Date
2013-01-01    15153419.52
2013-02-01    16524812.16
2013-03-01    28217082.24
2013-04-01    22522665.60
2013-05-01    32942280.96
Name: Sales, dtype: float64

In [272]:
best_pdq, best_seasonal_pdq, df = sarimax(finland_4_series, 'Finland', 4)

In [273]:
main_df = pd.concat([main_df, df])

## 'main_df' contains all the predicted values for all the combinations of Country and Product_ID

In [274]:
main_df.shape

(396, 5)

In [276]:
# main_df.to_csv('main_df.csv')

## Read test data and preprocess.

In [281]:
test_data = pd.read_csv(PATH_TO_test_data)
test_data.drop(['S_No', 'Sales'], axis=1, inplace=True)
test_data['Day'] = 1
test_data['Date'] = pd.to_datetime(test_data[['Year', 'Month', 'Day']])

In [283]:
test_data.head()

,Year,Month,Product_ID,Country,Day,Date
0,2016,4,1,Argentina,1,2016-04-01
1,2016,5,1,Argentina,1,2016-05-01
2,2016,6,1,Argentina,1,2016-06-01
3,2016,7,1,Argentina,1,2016-07-01
4,2016,8,1,Argentina,1,2016-08-01


In [358]:
duplicate_test_data = test_data

In [359]:
duplicate_test_data.head()

,Year,Month,Product_ID,Country,Day,Date
0,2016,4,1,Argentina,1,2016-04-01
1,2016,5,1,Argentina,1,2016-05-01
2,2016,6,1,Argentina,1,2016-06-01
3,2016,7,1,Argentina,1,2016-07-01
4,2016,8,1,Argentina,1,2016-08-01


In [360]:
duplicate_test_data.shape

(105, 6)

In [361]:
duplicate_main_df = main_df

In [362]:
duplicate_main_df = duplicate_main_df.reset_index()
duplicate_main_df.rename(columns = {'index': 'Date'}, inplace=True)

In [363]:
duplicate_main_df.head()

,Date,lower Sales,upper Sales,forcast,Country,Product_ID
0,2016-04-01,-2.204692e+07,2.563011e+07,1.791594e+06,Argentina,1
1,2016-05-01,-2.744635e+07,4.622482e+07,9.389230e+06,Argentina,1
2,2016-06-01,-2.217545e+07,5.842786e+07,1.812621e+07,Argentina,1
3,2016-07-01,-4.976126e+07,4.613342e+07,-1.813923e+06,Argentina,1
4,2016-08-01,-7.188802e+07,3.595239e+07,-1.796781e+07,Argentina,1


## Merge test data and main dataframe.

In [364]:
duplicate_test_data = pd.merge(duplicate_test_data, duplicate_main_df, on=['Product_ID', 'Country', 'Date'], how='left')

In [365]:
duplicate_test_data.head()

,Year,Month,Product_ID,Country,Day,Date,lower Sales,upper Sales,forcast
0,2016,4,1,Argentina,1,2016-04-01,-2.204692e+07,2.563011e+07,1.791594e+06
1,2016,5,1,Argentina,1,2016-05-01,-2.744635e+07,4.622482e+07,9.389230e+06
2,2016,6,1,Argentina,1,2016-06-01,-2.217545e+07,5.842786e+07,1.812621e+07
3,2016,7,1,Argentina,1,2016-07-01,-4.976126e+07,4.613342e+07,-1.813923e+06
4,2016,8,1,Argentina,1,2016-08-01,-7.188802e+07,3.595239e+07,-1.796781e+07


In [366]:
duplicate_test_data.drop(columns=['Day', 'Date', 'lower Sales', 'upper Sales'], inplace=True)

In [367]:
duplicate_test_data.head()

,Year,Month,Product_ID,Country,forcast
0,2016,4,1,Argentina,1.791594e+06
1,2016,5,1,Argentina,9.389230e+06
2,2016,6,1,Argentina,1.812621e+07
3,2016,7,1,Argentina,-1.813923e+06
4,2016,8,1,Argentina,-1.796781e+07


In [368]:
duplicate_test_data.shape

(105, 5)

In [369]:
duplicate_test_data = duplicate_test_data.rename(columns={'forcast': 'Sales'})

In [370]:
duplicate_test_data.head()

,Year,Month,Product_ID,Country,Sales
0,2016,4,1,Argentina,1.791594e+06
1,2016,5,1,Argentina,9.389230e+06
2,2016,6,1,Argentina,1.812621e+07
3,2016,7,1,Argentina,-1.813923e+06
4,2016,8,1,Argentina,-1.796781e+07


In [371]:
test_data_s_no = pd.read_csv(PATH_TO_test_data)

In [372]:
test_data_s_no.head()

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,NaN
1,79074,2016,5,1,Argentina,NaN
2,79075,2016,6,1,Argentina,NaN
3,79076,2016,7,1,Argentina,NaN
4,79077,2016,8,1,Argentina,NaN


In [373]:
duplicate_test_data['S_No'] = test_data_s_no.S_No

In [374]:
duplicate_test_data['Sales'] = duplicate_test_data['Sales'].abs()

In [375]:
duplicate_test_data = duplicate_test_data[['S_No', 'Year', 'Month', 'Product_ID', 'Country', 'Sales']]

In [376]:
duplicate_test_data.head()

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,1.791594e+06
1,79074,2016,5,1,Argentina,9.389230e+06
2,79075,2016,6,1,Argentina,1.812621e+07
3,79076,2016,7,1,Argentina,-1.813923e+06
4,79077,2016,8,1,Argentina,-1.796781e+07


In [377]:
duplicate_test_data.to_csv('yds_submission2018.csv', index=False)